In [1]:
from lblcrn.bulk_crn import *
from lblcrn.spec.crn import *
from lblcrn.experiments.xps_io import *
from lblcrn.experiments.solution_system import *
from lblcrn.experiments.tp_correlation import *

import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import seaborn as sb
import scipy.signal as signal
from mpl_toolkits.axes_grid1 import ImageGrid

In [2]:
sm = SpeciesManager()

y1 = sm.sp('y1', Orbital('1s', 535.0))
x2 = sm.sp('x2', Orbital('1s', 532.2))
x3 = sm.sp('x3', Orbital('1s', 530.9))
x4 = sm.sp('x4', Orbital('1s', 530.0))
x53 = sm.sp('x53', Orbital('1s', 531.6))
x54 = sm.sp('x54', Orbital('1s', 531.6))
x6 = sm.sp('x6', Orbital('1s', 533.2))
x7 = sm.sp('x7', Orbital('1s', 535.0))

multipliers = [0.1, 0.5, 1, 5, 10]

# constants=[0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.116930, 0.152889, 3.270343, 0.329519, 1.111032]
highp_hight = XPSInitializationData(
                'High P, High T',
                '../data/ag_100e-3_300.txt',
                573.15,
                100e-3
            )

# constants=[0.000039, 0.000017, 6.220646, 0.1607555, 0.299507, 0.167130, 0.794455, 0.000006, 0.300480, 0.127713, 0.000013, 0.394500, 0.000960, 76.964514]
lowp_rt = XPSInitializationData(
                'Low P,  RT',
                '../data/ag_1e-5_25.txt',
                298.15,
                1e-5
            )

init_data = [highp_hight, lowp_rt]

tp_correlator = XPSTPCorrelator(
    '../../data/H2O.01.out',
    573.15,
    100e-3,
    [6, 8, 9, 11],
    [0, 1, 7, 10],
    init_constants = [0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.116930, 0.152889, 3.270343, 0.329519, 1.111032],
)

ZPE: 11.583
H: {298.15: 2.371, 323.15: 2.571, 348.15: 2.771, 373.15: 2.972, 398.15: 3.174, 423.15: 3.377, 448.15: 3.581, 473.15: 3.787, 498.15: 3.993, 523.15: 4.202, 548.15: 4.412, 573.15: 4.624, 598.15: 4.838, 623.15: 5.054, 648.15: 5.272, 673.15: 5.492, 698.15: 5.715, 723.15: 5.939, 748.15: 6.166, 773.15: 6.395, 798.15: 6.626}
S: {298.15: 45.448, 323.15: 46.092, 348.15: 46.689, 373.15: 47.246, 398.15: 47.77, 423.15: 48.264, 448.15: 48.733, 473.15: 49.179, 498.15: 49.605, 523.15: 50.014, 548.15: 50.407, 573.15: 50.785, 598.15: 51.15, 623.15: 51.504, 648.15: 51.847, 673.15: 52.18, 698.15: 52.504, 723.15: 52.82, 748.15: 53.128, 773.15: 53.429, 798.15: 53.724}


In [3]:
def rsys_generator(scaled):
    rsys = RxnSystem(
        Rxn(x4 + y1, x54, scaled[0]), # 0
        Rxn(x3 + y1, x53, scaled[1]), # 1
        Rxn(x54, x3 + x3, scaled[2]), # 2
        Rxn(x3 + x3, x54, scaled[3]), # 3
        Rxn(x53, x2 + x3, scaled[4]), # 4
        Rxn(x54, x2 + x4, scaled[5]), # 5
        Rxn(x2, y1, scaled[6]), # 6
        Rxn(y1, x2, scaled[7]), # 7
        Rxn(x53, y1 + x3, scaled[8]), # 8
        Rxn(x54, x4 + y1, scaled[9]), # 9
        Rxn(x53 + y1, x6, scaled[10]), # 10
        Rxn(x6, x53 + y1, scaled[11]), # 11
        Rxn(x4 + x4, x7, scaled[12]), # 12
        Rxn(x7, x4 + x4, scaled[13]), # 13
        Conc(y1, 1.0),
        Conc(x4, 0.25),
        sm
    )
    return rsys

In [4]:
runner = XPSSystemRunner(rsys_generator, 10000, init_data, multipliers, tp_correlator=tp_correlator)

[0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 3.270343, 0.329519, 1.111032]
[1.100819025776275e-10, 7.914500604347608e-11, 2.023611, 0.494166, 0.628183, 0.501633, 9191380834.987804, 5.439893774553444e-11, 6317530167.3277855, 4542070966.455276, 7.125721133824273e-11, 7016826248.69516, 0.329519, 1.111032]


In [5]:
runner.initializer_data[0].constants

[0.236191,
 0.169813,
 2.023611,
 0.494166,
 0.628183,
 0.501633,
 4.283841,
 0.116718,
 2.944421,
 2.11693,
 0.152889,
 3.270343,
 0.329519,
 1.111032]

In [6]:
runner.initializer_data[1].constants
# Should be:
# [0.000039, 0.000017, 6.220646, 0.1607555, 0.299507, 0.167130, 0.794455, 0.000006, 0.300480, 0.127713, 0.000013, 0.394500, 0.000960, 76.964514]


[1.100819025776275e-10,
 7.914500604347608e-11,
 2.023611,
 0.494166,
 0.628183,
 0.501633,
 9191380834.987804,
 5.439893774553444e-11,
 6317530167.3277855,
 4542070966.455276,
 7.125721133824273e-11,
 7016826248.69516,
 0.329519,
 1.111032]

In [9]:
runner.simulate(0)

Solved for (0, 0)
[0.023619100000000004, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 3.270343, 0.329519, 1.111032]


Solved for (0, 1)
[0.1180955, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 3.270343, 0.329519, 1.111032]


Solved for (0, 2)
[0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 3.270343, 0.329519, 1.111032]


Solved for (0, 3)
[1.180955, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 3.270343, 0.329519, 1.111032]


Solved for (0, 4)
[2.36191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 3.270343, 0.329519, 1.111032]


Solved for (1, 0)
[0.236191, 0.0169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 3.270343, 0.329519, 1.111032]


Solved for (1, 1)
[0.236191

Solved for (10, 1)
[0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.0764445, 3.270343, 0.329519, 1.111032]


Solved for (10, 2)
[0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 3.270343, 0.329519, 1.111032]


Solved for (10, 3)
[0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.764445, 3.270343, 0.329519, 1.111032]


Solved for (10, 4)
[0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 1.52889, 3.270343, 0.329519, 1.111032]


Solved for (11, 0)
[0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 0.3270343, 0.329519, 1.111032]


Solved for (11, 1)
[0.236191, 0.169813, 2.023611, 0.494166, 0.628183, 0.501633, 4.283841, 0.116718, 2.944421, 2.11693, 0.152889, 1.6351715, 0.329519, 1.111032]


Solved for (11, 2)
[0.236191, 0.

In [11]:
runner.simulate(1)

KeyboardInterrupt: 